In [117]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import requests
import time
from youtube_transcript_api import YouTubeTranscriptApi
import re
from pytube import YouTube
import ssl

In [118]:
AUDIO_DIR = "audio"
ssl._create_default_https_context = ssl._create_unverified_context

In [119]:
def get_video_links(driver, channel_url):
    driver.get(channel_url)
    # TODO: stop this from being harcoded
    time.sleep(4)
    video_data = []

    # # Scroll to load videos (modify as necessary to ensure page loads)
    # for _ in range(5):
    #     driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    #     time.sleep(2)  # adjust as needed

    # Extract the channel name from the channel URL using regex
    pattern = r"youtube\.com\/@([^\/]+)\/videos"
    match = re.search(pattern, channel_url)
    channel_name = match.group(1) if match else "Unknown Channel"
    # print("channel_name: ", channel_name)

    # TODO: fix this
    time.sleep(4)

    # Capture existing video titles without waiting for all to be present
    videos = driver.find_elements(By.ID, 'video-title-link')
    if len(videos) == 0:
        print("shiiiii no vids!")
    
    # Collect data for the first 5 videos
    for vid in videos[:5]:
        print("in the loop!")
        video_title = vid.get_attribute('title')
        link = vid.get_attribute('href')
        # print("video_link: ", link)
        # print("video_title: ", video_title)
        video_data.append({"link": link, "channel": channel_name, "episode": video_title})
    
    return video_data

In [120]:
def get_transcript(id: str) -> None:
    transcript = YouTubeTranscriptApi.get_transcript(
        id, languages=('en', 'en-US', 'en-GB'))
    return transcript

In [121]:
def download_audio(url, file_name):
    print("in download audio, url is: ", url)
    try:
        yt = YouTube(url)
    except Exception as e:
        print(f"Error making youtube object: {e}")
    try:
        t=yt.streams.filter(only_audio=True).all()
        #t[0].download(AUDIO_DIR + f"/{file_name}.mp3")
        t[0].download("test")
    except Exception as e:
        print(f"Download error: {e}")
    print('Task Completed!')

In [122]:
def main():
    driver = webdriver.Chrome()

    channels = [
        "https://www.youtube.com/@a16z/videos",
        "https://www.youtube.com/@Freakonomics/videos",
        # add more channels as needed
    ]

    all_results = []
    for channel in channels:
        new_video_list = get_video_links(driver, channel)
        for video in new_video_list:
            all_results.append(video)

    for video in all_results:
        id = video['link'].split("v=")[1]
        channel = video['channel']
        episode = video['episode']
        video['transcript'] = get_transcript(id)
        download_audio(video['link'], f"{channel}_{episode}")

    driver.quit()
    return pd.DataFrame(all_results)

# Run the script
df = main()
print(df)

in the loop!
in the loop!
in the loop!
in the loop!
in the loop!
in the loop!
in the loop!
in the loop!
in download audio, url is:  https://www.youtube.com/watch?v=dX7d6bRJI9k
Download error: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>
Task Completed!
in download audio, url is:  https://www.youtube.com/watch?v=xgf2FD62j3g
Download error: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>
Task Completed!
in download audio, url is:  https://www.youtube.com/watch?v=HQV9s_JgNZA
Download error: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>
Task Completed!
in download audio, url is:  https://www.youtube.com/watch?v=BZUMZl4OsPI
Download error: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certifi

In [123]:
df.to_csv('ballsack.csv', index=False)